In [4]:
import os, glob, cv2, numpy as np
from pathlib import Path
import shutil
import random

random.seed(42)

SRC = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset")
DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")

# 0) 기본 디렉토리 준비
for sub in ["images/train","images/val","labels/train","labels/val"]:
    (DST/sub).mkdir(parents=True, exist_ok=True)

def copy_pair(stem, split):
    # 이미지 확장자 탐색
    for ext in [".jpg",".png",".jpeg",".JPG",".PNG",".JPEG"]:
        ip = SRC / "images" / split / f"{stem}{ext}"
        if ip.exists():
            break
    else:
        return False
    lp = SRC / "labels" / split / f"{stem}.txt"
    if not lp.exists():
        return False

    # 원본 복사(이미 존재하면 건너뜀)
    img_dst = DST / "images" / split / ip.name
    lab_dst = DST / "labels" / split / f"{stem}.txt"
    if not img_dst.exists(): shutil.copy2(ip, img_dst)
    if not lab_dst.exists(): shutil.copy2(lp, lab_dst)
    return True

# 1) train/val 원본 먼저 그대로 복사
train_labels = sorted(glob.glob(str(SRC / "labels/train/*.txt")))
val_labels   = sorted(glob.glob(str(SRC / "labels/val/*.txt")))

n_copied_train = 0
for lp in train_labels:
    stem = Path(lp).stem
    if copy_pair(stem, "train"):
        n_copied_train += 1

n_copied_val = 0
for lp in val_labels:
    stem = Path(lp).stem
    if copy_pair(stem, "val"):
        n_copied_val += 1

print(f"Copied train={n_copied_train}, val={n_copied_val}")

# 2) 저화질 변형을 train에만 추가 생성
def read_img_by_stem(stem):
    for ext in [".jpg",".png",".jpeg",".JPG",".PNG",".JPEG"]:
        p = SRC / "images/train" / f"{stem}{ext}"
        if p.exists():
            img = cv2.imread(str(p))
            if img is not None:
                return img, p.suffix
    return None, None

def save_aug(img, out_path):
    # 확장자에 맞춰 저장
    out_path.parent.mkdir(parents=True, exist_ok=True)
    ext = out_path.suffix.lower()
    if ext in [".jpg",".jpeg"]:
        cv2.imwrite(str(out_path), img, [cv2.IMWRITE_JPEG_QUALITY, 95])
    elif ext == ".png":
        cv2.imwrite(str(out_path), img, [cv2.IMWRITE_PNG_COMPRESSION, 3])
    else:
        cv2.imwrite(str(out_path), img)

def aug_blur(img):
    # 가우시안 블러 (odd 커널)
    k = random.choice([3,5,7])
    return cv2.GaussianBlur(img, (k,k), 0)

def aug_jpeg_compress(img):
    # JPEG 품질 낮게 재인코딩
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), random.choice([30,40,50,60,70])]
    _, enc = cv2.imencode(".jpg", img, encode_param)
    dec = cv2.imdecode(enc, cv2.IMREAD_COLOR)
    return dec

def aug_downscale_upscale(img):
    # 해상도 낮춘 뒤 다시 원래 크기로 업스케일 (aliasing/blur 유도)
    h, w = img.shape[:2]
    scale = random.choice([0.5, 0.6, 0.7])
    small = cv2.resize(img, (int(w*scale), int(h*scale)), interpolation=cv2.INTER_AREA)
    back = cv2.resize(small, (w, h), interpolation=cv2.INTER_LINEAR)
    return back

# 몇 %를 증강으로 추가할지 (원본 train의 일부만 선택해서 3종류 증강을 각각 1회 추가)
AUG_RATIO = 0.5  # train 샘플의 50%를 뽑아 각 변형을 추가(과하면 과적합/편향 가능 → 0.3~0.7 사이에서 조절)
chosen = random.sample(train_labels, k=int(len(train_labels)*AUG_RATIO))
print("to augment:", len(chosen))

added = {"blur":0, "jpeg":0, "downup":0}
for i, lp in enumerate(chosen):
    stem = Path(lp).stem
    img, ext = read_img_by_stem(stem)
    if img is None: 
        continue
    # 3가지 변형 각각 1장씩 추가 (파일명 접미사로 구분)
    variants = {
        "blur": aug_blur(img),
        "jpeg": aug_jpeg_compress(img),
        "downup": aug_downscale_upscale(img),
    }
    for tag, aug_img in variants.items():
        out_img = DST / "images/train" / f"{stem}__{tag}{ext if ext.lower() in ['.jpg','.jpeg','.png'] else '.jpg'}"
        out_lab = DST / "labels/train" / f"{stem}__{tag}.txt"
        # 이미지 저장
        save_aug(aug_img, out_img)
        # 라벨 복사 (동일 박스)
        shutil.copy2(lp, out_lab)
        added[tag] += 1

print("added (per transform):", added)
print("Done →", DST)


Copied train=1664, val=416
to augment: 832
added (per transform): {'blur': 832, 'jpeg': 832, 'downup': 832}
Done → /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2


In [5]:
import os
from pathlib import Path

DST = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")

def count_labels_images(split="train"):
    img_dir = DST / "images" / split
    lab_dir = DST / "labels" / split

    # 지원 확장자
    exts = [".jpg", ".jpeg", ".png", ".JPG", ".JPEG", ".PNG"]

    # 이미지 목록
    images = [p for p in img_dir.iterdir() if p.suffix in exts]
    labels = [p for p in lab_dir.iterdir() if p.suffix == ".txt"]

    # stem (확장자 제거 이름) 기준 매칭
    label_stems = {p.stem for p in labels}
    no_label = [p for p in images if p.stem not in label_stems]

    print(f"[{split}]")
    print(f"  Images: {len(images)}")
    print(f"  Labels: {len(labels)}")
    print(f"  Images w/o label: {len(no_label)}")
    if no_label:
        print("  Missing label files:")
        for p in no_label[:10]:  # 10개만 미리보기
            print("   -", p.name)
    print()

# train / val 각각 출력
count_labels_images("train")
count_labels_images("val")


[train]
  Images: 4160
  Labels: 4160
  Images w/o label: 0

[val]
  Images: 416
  Labels: 416
  Images w/o label: 0



In [6]:
import os
from pathlib import Path

DATASET = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")

def count_empty_labels(split="train"):
    labels_dir = DATASET / "labels" / split
    empty, non_empty = 0, 0
    for f in labels_dir.glob("*.txt"):
        with open(f, "r") as fp:
            lines = [l.strip() for l in fp.readlines() if l.strip()]
        if len(lines) == 0:   # 바운딩 박스 없음
            empty += 1
        else:
            non_empty += 1
    return empty, non_empty

for split in ["train", "val"]:
    empty, non_empty = count_empty_labels(split)
    total = empty + non_empty
    print(f"[{split}] total={total}, empty={empty}, with_boxes={non_empty}, empty_ratio={empty/total:.2%}")


[train] total=4160, empty=1534, with_boxes=2626, empty_ratio=36.88%
[val] total=416, empty=148, with_boxes=268, empty_ratio=35.58%


In [7]:
import os, random
from pathlib import Path

ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")
L_T = ROOT/"labels/train"
I_T = ROOT/"images/train"

AUG_TAGS = ("__blur", "__jpeg", "__downup")
EXTS = [".jpg",".jpeg",".png",".JPG",".JPEG",".PNG"]

def has_image(stem):
    for e in EXTS:
        p = I_T/f"{stem}{e}"
        if p.exists():
            return p
    return None

aug_empty = []
for lp in L_T.glob("*.txt"):
    stem = lp.stem
    if any(tag in stem for tag in AUG_TAGS):  # 증강본만
        if lp.stat().st_size == 0:           # 빈 라벨
            aug_empty.append(stem)

print("augmented empty labels:", len(aug_empty))
deleted = 0
for stem in aug_empty:
    # 라벨 삭제
    (L_T/f"{stem}.txt").unlink(missing_ok=True)
    # 이미지 삭제
    ip = has_image(stem)
    if ip: ip.unlink()
    deleted += 1

print("deleted augmented-empty pairs:", deleted)


augmented empty labels: 930
deleted augmented-empty pairs: 930


In [8]:
import os, random
from pathlib import Path

random.seed(42)

ROOT = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")
L_T = ROOT/"labels/train"
I_T = ROOT/"images/train"
EXTS = [".jpg",".jpeg",".png",".JPG",".JPEG",".PNG"]

def has_image(stem):
    for e in EXTS:
        p = I_T/f"{stem}{e}"
        if p.exists():
            return p
    return None

labels = list(L_T.glob("*.txt"))
empty = [p for p in labels if p.stat().st_size == 0]
nonempty = [p for p in labels if p.stat().st_size > 0]

TOTAL = len(labels)
TARGET_RATIO = 0.15  # 원하는 배경 비율
target_keep = int(TOTAL * TARGET_RATIO)

print(f"current empty={len(empty)} / total={TOTAL} ({len(empty)/TOTAL:.2%})")
print("target empty keep:", target_keep)

to_delete = []
if len(empty) > target_keep:
    random.shuffle(empty)
    to_delete = empty[target_keep:]  # 초과분 삭제
else:
    print("No need to prune; empty <= target.")

deleted = 0
for lp in to_delete:
    stem = lp.stem
    # 라벨 삭제
    lp.unlink(missing_ok=True)
    # 이미지 삭제
    ip = has_image(stem)
    if ip: ip.unlink()
    deleted += 1

print("deleted empty pairs:", deleted)

# 캐시 갱신을 위해 cache 파일 삭제
for cache in [ROOT/"labels/train.cache", ROOT/"labels/val.cache"]:
    if cache.exists():
        cache.unlink()
        print("removed cache:", cache)


current empty=604 / total=3230 (18.70%)
target empty keep: 484
deleted empty pairs: 120


In [9]:
import os
from pathlib import Path

DATASET = Path("/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2")

def count_empty_labels(split="train"):
    labels_dir = DATASET / "labels" / split
    empty, non_empty = 0, 0
    for f in labels_dir.glob("*.txt"):
        with open(f, "r") as fp:
            lines = [l.strip() for l in fp.readlines() if l.strip()]
        if len(lines) == 0:   # 바운딩 박스 없음
            empty += 1
        else:
            non_empty += 1
    return empty, non_empty

for split in ["train", "val"]:
    empty, non_empty = count_empty_labels(split)
    total = empty + non_empty
    print(f"[{split}] total={total}, empty={empty}, with_boxes={non_empty}, empty_ratio={empty/total:.2%}")


[train] total=3110, empty=484, with_boxes=2626, empty_ratio=15.56%
[val] total=416, empty=148, with_boxes=268, empty_ratio=35.58%


In [ ]:
from ultralytics import YOLO

DATA_YAML = "/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/data2.yaml"
m = YOLO("yolov8s.pt")

m.train(
    data=DATA_YAML,
    epochs=300, patience=50,
    imgsz=896, batch=8, device=0,
    rect=True,
    mosaic=0.8, close_mosaic=15,
    erasing=0.3, mixup=0.0, copy_paste=0.0,
    hsv_h=0.02, hsv_s=0.6, hsv_v=0.5,
    cls=1.0,                 # 분류 손실 가중치만 유지
    cos_lr=True, cache=True,
    project="/home/dw/ws_job_msislab/Golf_Project/runs_yolo",
    name="golf_yv8s_hiLo_aug",
)



Ultralytics 8.3.175 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 7932MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=15, cls=1.0, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=True, cutmix=0.0, data=/home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/data2.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=30, erasing=0.3, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.02, hsv_s=0.6, hsv_v=0.5, imgsz=896, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8s.pt, momentum=0.937, mosaic=0.8, multi_scale=False, name=golf_yv8s_hiLo_aug2, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, pa

train: Scanning /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2/labels/train... 4160 images, 1534 backgrounds, 0 corrupt: 100%|██████████| 4160/4160 [00:01<00:00, 4117.22it/s]

train: New cache created: /home/dw/ws_job_msislab/Golf_Project/data/Data_20250725/yolo_dataset2/labels/train.cache


WARNING ⚠️ cache='ram' may produce non-deterministic training results. Consider cache='disk' as a deterministic alternative if your disk space allows.


train: Caching images (7.5GB RAM): 100%|██████████| 4160/4160 [00:18<00:00, 221.93it/s]

...

      17/30      3.22G      1.575      3.634      1.328          8        896: 100%|██████████| 520/520 [01:04<00:00,  8.09it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:02<00:00, 11.00it/s]

                   all        416        600      0.516      0.545      0.524      0.244



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      18/30      3.22G      1.584      3.721      1.358          4        896:  22%|██▏       | 116/520 [00:14<00:50,  8.01it/s]


KeyboardInterrupt: 